In [5]:
!pip install transformers[ja,sentencepiece,torch] > /dev/null

In [6]:
!pip install fugashi unidic_lite > /dev/null

In [7]:
from transformers import pipeline

## 文書分類

In [8]:
text_classification_pipeline = pipeline(
    model = "llm-book/bert-base-japanese-v3-marc_ja"
)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [9]:
def nega_posi_judge(text: str) -> None:
    """
    テキストを受け取りネガティブかポジティブか判定し、結果を出力する関数

    受け取ったテキストを感情分類モデルに入力し、最も確からしい感情ラベルと
    その確率をパーセンテージで出力する。

    Args:
        text (str): 判定したいテキスト

    Returns:
        None
    """
    result = text_classification_pipeline(text)
    print(f"このテキストは{result[0]['score']*100:.1f}%の確率で{result[0]['label']}な文章です")

In [10]:
posi_text = "世界には言葉が分からなくても感動する音楽がある。"

nega_posi_judge(posi_text)

このテキストは99.9%の確率でpositiveな文章です


In [11]:
nega_text = "世界には言葉が出ないほどひどい音楽がある。"

nega_posi_judge(nega_text)

このテキストは98.6%の確率でnegativeな文章です


## 自然言語推論

In [12]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "2人の男性がジェット機を見ています。"
entailment_text = "ジェット機を見ている2人がいます。"

# textとentailment_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": entailment_text}))

Device set to use cpu


{'label': 'entailment', 'score': 0.9977602958679199}


In [13]:
contradiction_text = "2人の男性が飛んでいます"

print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

{'label': 'contradiction', 'score': 0.9423864483833313}


In [14]:
neutral_text = "2人の男性が、白い飛行機を眺めています。"

print(nli_pipeline({"text": text, "text_pair": neutral_text}))

{'label': 'neutral', 'score': 0.9977497458457947}
